<a href="https://colab.research.google.com/github/iwatake2222/colaboratory_study/blob/master/object_detection/beer_detection/beer_detection_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object detection model training
- dataset: beer (tfrecord)
- pretrained model: faster_rcnn_resnet101_coco, ssd_mobilenet_v2_quantized_300x300_coco
  - config files (modify and upload to google drive)
    - https://github.com/tensorflow/models/blob/master/research/object_detection/samples/configs/faster_rcnn_resnet101_coco.config
    - https://github.com/tensorflow/models/blob/master/research/object_detection/samples/configs/ssd_mobilenet_v2_quantized_300x300_coco.config
  - pretrained models (https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md )
    - http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz
    - http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz

- note
  - need to decrease the number of batch_size in config file, otherwise OOM occurs
  - find `### MODIFY ###` in the config file and scripts


# Preparation
## Setting for environment


In [0]:
from google.colab import drive 
drive.mount('/content/drive')
!ls

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
!pip install pycocotools
!pip list

!git clone --depth 1 https://github.com/tensorflow/models
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!pip install .
%cd /content

# Modify python path and check script
%set_env PYTHONPATH=$PYTHONPATH:models/research:models/research/slim
!echo $PYTHONPATH
!python models/research/object_detection/builders/model_builder_test.py

# Setting for training

## Get training data

In [0]:
%%bash
### MODIFY ### (depending on dataset)
FILE_ID=1-3M4Thwd3tH0XYglvaKwnpakjFrgR2gr
FILE_NAME=dataset_beer.tar.gz

curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=${FILE_ID}" > /dev/null
CODE="$(awk '/_warning_/ {print $NF}' /tmp/cookie)"  
curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=${CODE}&id=${FILE_ID}" -o ${FILE_NAME}
tar -xvf ${FILE_NAME}

In [196]:
### MODIFY ### (if needed)
DATASET_ORG_DIR = "/content/dataset_beer"
DATASET_TRAIN = "/content/dataset/train"
DATASET_EVAL = "/content/dataset/eval"

!rm -rf  "$DATASET_TRAIN" && mkdir -p "$DATASET_TRAIN"
!rm -rf  "$DATASET_EVAL" && mkdir -p "$DATASET_EVAL"
!cp "$DATASET_ORG_DIR/tf_label_map.pbtxt" "$DATASET_TRAIN/../."

import random
import glob
import shutil

img_list = glob.glob(DATASET_ORG_DIR + "/tfrecord/*.tfrecord")
files_train = set(random.sample(
  img_list,
  int(len(img_list)*.7)
))
files_eval = set(img_list) - file_to_dir1
for f in files_train:
  shutil.copy(f, DATASET_TRAIN)

for f in files_eval:
  shutil.copy(f, DATASET_EVAL)

!ls -1 "$DATASET_TRAIN" | wc -l
!ls -1 "$DATASET_EVAL" | wc -l

137
60


## Get pretrained model

In [0]:
### MODIFY ### (depending on pretrained model)
# PRETRAINED_MODEL_URL =  "http://storage.googleapis.com/download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz"
PRETRAINED_MODEL_URL =  "http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz"
PRETRAINED_MODEL_DIR = "/content/dataset/pretrained_model"

!wget "$PRETRAINED_MODEL_URL" -O pretrained_model.tar.gz
!rm -rf "$PRETRAINED_MODEL_DIR" && mkdir -p "$PRETRAINED_MODEL_DIR"
!tar -xvf pretrained_model.tar.gz -C "$PRETRAINED_MODEL_DIR" --strip-components 1
!ls "$PRETRAINED_MODEL_DIR"

## Get config file
https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs

In [0]:
### MODIFY ### (depending on training)
# TRAIN_DIR = "/content/drive/My Drive/training/beer_detection/faster_rcnn_resnet101_coco_11_06_2017_20191230"
# ORIGINAL_CONFIG_FILE = "faster_rcnn_resnet101_coco.config"
# CONFIG_FILE = "/faster_rcnn_resnet101_beer.config"
TRAIN_DIR = "/content/drive/My Drive/training/beer_detection/ssd_mobilenet_v2_quantized_300x300_coco_20200104"
ORIGINAL_CONFIG_FILE = "ssd_mobilenet_v2_quantized_300x300_coco.config"
CONFIG_FILE = "/ssd_mobilenet_v2_quantized_300x300_beer.config"

!mkdir -p "$TRAIN_DIR"
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/samples/configs/"$ORIGINAL_CONFIG_FILE" -P "$TRAIN_DIR"
!cp "$TRAIN_DIR/$ORIGINAL_CONFIG_FILE" "$TRAIN_DIR/$CONFIG_FILE" 
!cp "$DATASET_ORG_DIR/tf_label_map.pbtxt" "$TRAIN_DIR/."

## Edit config file


### faster_rcnn_resnet101_beer.config

In [0]:
'''
# Faster R-CNN with Resnet-101 (v1), configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    ### MODIFY ###
    num_classes: 10
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet101'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        ### MODIFY ###
        max_total_detections: 100
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 900000
            learning_rate: .00003
          }
          schedule {
            step: 1200000
            learning_rate: .000003
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  ### MODIFY ###
  fine_tune_checkpoint: "/content/dataset/pretrained_model/model.ckpt"
  from_detection_checkpoint: true
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}

train_input_reader: {
  ### MODIFY ###
  tf_record_input_reader {
    input_path: "/content/dataset/train/*.tfrecord"
  }
  label_map_path: "/content/dataset/tf_label_map.pbtxt"
}

eval_config: {
  num_examples: 8000
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  max_evals: 10
}

eval_input_reader: {
  ### MODIFY ###
  tf_record_input_reader {
    input_path: "/content/dataset/eval/*.tfrecord"
  }
  label_map_path: "/content/dataset/tf_label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

'''

### ssd_mobilenet_v2_quantized_300x300_beer.config

In [0]:
'''
# Quantized trained SSD with Mobilenet v2 on MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    ### MODIFY ###
    num_classes: 10
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        dropout_keep_probability: 0.8
        kernel_size: 1
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {
            train: true,
            scale: true,
            center: true,
            decay: 0.9997,
            epsilon: 0.001,
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.99
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 3
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  ### MODIFY ###
  batch_size: 10
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.004
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }
  ### MODIFY ###
  fine_tune_checkpoint: "/content/dataset/pretrained_model/model.ckpt"
  fine_tune_checkpoint_type:  "detection"
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  ### MODIFIED ###
  tf_record_input_reader {
    input_path: "/content/dataset/train/*.tfrecord"
  }
  label_map_path: "/content/dataset/tf_label_map.pbtxt"
}

eval_config: {
  num_examples: 8000
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  max_evals: 10
}

eval_input_reader: {
  ### MODIFIED ###
  tf_record_input_reader {
    input_path: "/content/dataset/eval/*.tfrecord"
  }
  label_map_path: "/content/dataset/tf_label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

graph_rewriter {
  quantization {
    delay: 48000
    weight_bits: 8
    activation_bits: 8
  }
}
'''

# Training

In [0]:
!python models/research/object_detection/model_main.py \
    --pipeline_config_path="$TRAIN_DIR/$CONFIG_FILE" \
    --model_dir="$TRAIN_DIR" \
    --num_train_steps=10000 \
    --num_eval_steps=2000 \
    --alsologtostderr


# Export graph

In [0]:
# this doesn't work for tflite conversion
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path="$TRAIN_DIR/$CONFIG_FILE" \
    --trained_checkpoint_prefix="$TRAIN_DIR/model.ckpt-10000" \
    --output_directory="$TRAIN_DIR/exported_graphs"

In [0]:
# Use this instead for tflite conversion
!python models/research/object_detection/export_tflite_ssd_graph.py	\
    --input_type=image_tensor \
    --pipeline_config_path="$TRAIN_DIR/$CONFIG_FILE" \
    --trained_checkpoint_prefix="$TRAIN_DIR/model.ckpt-10000" \
    --output_directory="$TRAIN_DIR/exported_graphs" \
    --add_postprocessing_op=true



## Check model structure
to identify input and output nodes

In [0]:
Graph = tf.GraphDef()
# File = open(TRAIN_DIR + "/exported_graphs/frozen_inference_graph.pb","rb")
File = open(TRAIN_DIR + "/exported_graphs/tflite_graph.pb","rb")
Graph.ParseFromString(File.read())
for Layer in Graph.node:
  print(Layer.name)

## Convert to tflite

In [0]:
input_arrays = ["normalized_input_image_tensor"]
converter = tf.lite.TFLiteConverter.from_frozen_graph(
  TRAIN_DIR + "/exported_graphs/tflite_graph.pb",
  input_arrays,
  ["TFLite_Detection_PostProcess", "TFLite_Detection_PostProcess:1", "TFLite_Detection_PostProcess:2", "TFLite_Detection_PostProcess:3"],
  input_shapes={input_arrays[0]:[1,300,300,3]}
)

#converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.allow_custom_ops=True
converter.inference_input_type = tf.lite.constants.QUANTIZED_UINT8
# converter.inference_output_type = tf.lite.constants.QUANTIZED_UINT8
converter.quantized_input_stats = {input_arrays[0] : (128., 127.)}  # mean, std_dev ((quantized_input_value - mean_value) / std_dev_value.)
converter.default_ranges_stats = (-10, 10)
converter.inference_type = tf.lite.constants.QUANTIZED_UINT8

tflite_model = converter.convert()
open(TRAIN_DIR + "/exported_graphs/beer_mobilenet_v2_quantized_300x300.tflite", "wb").write(tflite_model)

### Convert for edge TPU

In [0]:
%%bash 
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
sudo apt-get update
sudo apt-get install edgetpu

In [0]:
!edgetpu_compiler "$TRAIN_DIR/exported_graphs/beer_mobilenet_v2_quantized_300x300.tflite"
!cp beer_mobilenet_v2_quantized_300x300_edgetpu.tflite "$TRAIN_DIR/exported_graphs/."